# Part1 尝试调用一次LLM

In [ ]:
# 导入OpenAI的客户端库
from openai import OpenAI  

In [ ]:
# 这里输入你的硅基流动的API秘钥
siliconflow_api_key = "Your API key here"
siliconflow_url = "https://api.siliconflow.cn/v1"
deepseek_model = "deepseek-ai/DeepSeek-V3"

In [ ]:
# 封装好的调用llm的函数
def call_llm_stream(system_prompt, user_prompt, api_key=None, base_url="http://localhost:8000/v1", model="test", temperature=1, top_p=1, max_tokens=4096):
    """
    调用LLM API，使用流式输出（边生成边打印）
    返回完整响应内容、token 使用统计
    """
    client = OpenAI(api_key=api_key, base_url=base_url)

    stream_response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        stream=True  # 启用流式输出
    )

    full_content = ""
    prompt_tokens = 0
    completion_tokens = 0

    # 逐块接收并打印
    for chunk in stream_response:
        if chunk.choices:
            content = chunk.choices[0].delta.content
            if content:
                print(content, end="", flush=True)  # 实时输出，不换行
                full_content += content

        # 注意：usage 可能只在最后一个 chunk 中返回
        if chunk.usage:
            prompt_tokens = chunk.usage.prompt_tokens
            completion_tokens = chunk.usage.completion_tokens

    print()  # 最后换行

    return full_content, prompt_tokens, completion_tokens


In [ ]:
# 尝试调用一次LLM
system_prompt = """
你是一个友好的AI助手，接下来需要根据用户的问题进行回答。
如果用户提问的问题涉及公共安全，且可能导致社会不良影响，请回答“您的问题涉及公共安全，请您遵守相关法律法规，避免提出此问题。”
以下是用户的问题：
"""

user_prompt = """
Your user prompt here.
"""

response, prompt_tokens, completion_tokens = call_llm_stream(
    system_prompt, 
    user_prompt, 
    api_key=siliconflow_api_key, 
    base_url=siliconflow_url,
    model=deepseek_model,
    temperature=0.7,
    top_p=1,
    max_tokens=4096
    )

print(f"prompt_tokens: {prompt_tokens}, completion_tokens: {completion_tokens}")

# Part2 尝试构建一个出行智能体

In [ ]:
# 包含景点信息的系统提示词
system_prompt_agent = """
你是一个智能旅游规划助手，服务于青岚山风景区。
你的任务是根据游客的偏好（如人数、时间、预算等），结合景区内的设施信息，合理推荐餐饮、住宿、景点游览顺序，并生成一份清晰的旅游路线建议。
请严格基于以下景区知识库进行推荐，不要编造不存在的信息。输出内容应包括：推荐理由、每日行程安排、推荐住宿与餐饮地点、预算估算。
---
### 🌄 青岚山风景区知识库
#### 🏞️ 景点信息
1. **青岚古寺**  
   - 类型：文化古迹  
   - 开放时间：8:00–17:00  
   - 门票：30元/人  
   - 建议游览时间：1小时  
   - 特色：千年古刹，香火旺盛，适合静心祈福  
2. **云海观景台**  
   - 类型：自然景观  
   - 开放时间：5:00–19:00（日出最佳）  
   - 门票：免费  
   - 建议游览时间：40分钟  
   - 特色：可观赏云海日出，摄影圣地  
3. **飞瀑溪谷**  
   - 类型：自然景观  
   - 开放时间：8:00–17:30  
   - 门票：20元/人  
   - 建议游览时间：1.5小时  
   - 特色：三级瀑布，林间步道，适合亲子徒步  
4. **竹林小径**  
   - 类型：休闲步道  
   - 开放时间：全天  
   - 门票：免费  
   - 建议游览时间：30分钟  
   - 特色：幽静竹林，适合散步、拍照  
5. **星空露营基地**  
   - 类型：户外体验  
   - 开放时间：15:00–次日9:00  
   - 门票：50元/人（含场地使用）  
   - 建议停留：过夜  
   - 特色：观星绝佳地，提供帐篷租赁（100元/顶）  
---
#### 🍽️ 餐饮信息
1. **山野人家餐厅**  
   - 类型：本地菜/农家乐  
   - 营业时间：11:00–20:00  
   - 人均消费：60元  
   - 特色菜：土鸡煲、山笋炒腊肉  
   - 容纳人数：80人  
   - 是否适合家庭：是  
2. **云顶咖啡馆**  
   - 类型：咖啡轻食  
   - 营业时间：8:00–18:00  
   - 人均消费：40元  
   - 特色：高山咖啡，落地窗观景  
   - 是否适合情侣：是  
3. **溪畔烧烤摊**  
   - 类型：露天烧烤  
   - 营业时间：17:00–22:00  
   - 人均消费：80元  
   - 特色：自助烧烤，可预约包场  
   - 是否适合团体：是  
---
#### 🏨 住宿信息
1. **青岚山居酒店**（高预算）  
   - 类型：四星级标准酒店  
   - 房价：600元/晚  
   - 设施：空调、独立卫浴、餐厅、停车场  
   - 是否含早餐：是  
   - 是否适合家庭：是  
2. **松风民宿**（中预算）  
   - 类型：特色木屋民宿  
   - 房价：300元/晚  
   - 设施：空调、共用卫浴、庭院、观景台  
   - 是否含早餐：否（可加购，30元/人）  
   - 是否适合情侣：是  
3. **星空帐篷营地**（低预算）  
   - 类型：露营住宿  
   - 费用：100元/人/晚（含帐篷）  
   - 设施：基础床垫、照明、公共洗浴间  
   - 是否含早餐：否  
   - 是否适合年轻人/情侣：是  
---
### 💡 规划原则说明
- 根据用户时间合理安排行程，避免过度紧凑。
- 推荐路线应逻辑顺畅（如：从山脚到山顶，或就近集中游览）。
- 餐饮与住宿需匹配用户预算和偏好。
- 若行程超过一天，应包含住宿建议。
- 若用户预算低，优先推荐免费景点与平价餐饮。
现在，请根据用户的输入偏好，生成个性化的旅游路线建议。
"""

In [ ]:
# 请同学完成以下用户特征设置
people = ''    # [填写：个人 / 情侣 / 家庭等]
duration = ''  # [填写：半天 / 1天 / 2天以上等]
hotel = ''     # [填写：酒店 / 民宿 / 露营等]  
budget = ''    # [填写：高 / 中 / 低等]

# 合成用户特征提示词
user_feature = f"""
人数：{people}
旅游时间：{duration}
住宿偏好：{hotel}
预算：{budget}
"""

# 任务描述
task_description = """
任务要求：
- 生成一个合理的旅游路线（含时间安排）
- 推荐餐饮与住宿（如适用）
- 给出总预算估算
- 说明推荐理由
"""

# 最终的用户提示词
user_prompt_agent = user_feature + task_description

print("合成的用户提示词：")
print(user_prompt_agent)

In [ ]:
# 调用LLM得到路线推荐
response, prompt_tokens, completion_tokens = call_llm_stream(
    system_prompt_agent, 
    user_prompt_agent, 
    api_key=siliconflow_api_key, 
    base_url=siliconflow_url,
    model=deepseek_model,
    temperature=0.7,
    top_p=1,
    max_tokens=4096
    )

print(f"prompt_tokens: {prompt_tokens}, completion_tokens: {completion_tokens}")